# Web Scrapping para obtener los datos de vecindarios en Toronto

### Importamos librerías necesarias

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

### 2. Obtenemos los datos de los vecindarios desde Wikipedia

Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia,&nbsp;https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,&nbsp;para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de&nbsp;

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_data = requests.get(url).text
raw_data = BeautifulSoup(wiki_data,'lxml')

Imprimimos la data para darle un vistazo

In [3]:
raw_data

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(/(?:^|;

### 3. Creando el marco de datos

Generamos el web scrapping adaptandolo a los siguientes criterios (pueden no estar en orden de acuerdo al procesamiento)

-   El marco de datos constará de tres columnas: CódigoPostal, Municipio y Vecindario
-   Procese únicamente las celdas que tengan un municipio asignado. Ignore o elimine las celdas con un municipio que no esté asignado
-   Puede existir más de un vecindario en un área de código postal. Por ejemplo, en la tabla de la página de Wikipedia, notará que M%A aparece dos veces y tiene dos vecindarios Harbourfront y Regent Park. Estas dos filas se combinarán en una fila con los vecindarios separados con una coma
-   Si una celda tiene un municipio pero no un vecindario, entonces el vecindario será el mismo que el municipio

In [12]:
#creamos el dataframe con las columnas solicitadas
column_names = ['Postalcode','Borough','Neighborhood'] 
data_toronto = pd.DataFrame(columns = column_names)

In [ ]:
# Es momento de leer y buscar la información de la página en Wikipedia
content = raw_data.find('div', class_='vector-page-toolbar')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    char = "(),"
    pattern  =  "[" + char + "]"
    for td in tr.find_all('td'):
        neighborhood=''
        postcode = td.text
        if len(td.find_all('i')) == 1: #si el municipio es not assigned el vecindario tmb esto servirá para limpiar después
            borough = td.find('i').text
            neighborhood = borough
        else:
            span = td.find('span')
            a_list = span.find_all('a')
            for index,a in enumerate(a_list):
                if index<=0:
                    borough = a.text
                else:
                    neighborhood = neighborhood+','+a.text #aqui se combinan los vecindarios desde la extracción de información


        neighborhood = neighborhood.lstrip(',')
        neighborhood = neighborhood.lstrip('(') 
        
        data_toronto.loc[len(data_toronto)] = {'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood}

In [7]:
#Hechemos un vistazo al dataframe creado
data_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"


In [8]:
#Eliminemos todos los codigos postales no assignados
clean_data = data_toronto[data_toronto['Borough'] != 'Not assigned']
clean_data = clean_data.dropna()

In [10]:
clean_data.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,25,103
top,M1B,North York,"Malvern,Rouge"
freq,1,25,1


In [11]:
#imprimimos el resultado para observar el numero de filas del marco de datos generado
clean_data.shape

(103, 3)